# <a id='toc1_'></a>[Projet 3 Préparez des données pour un organisme de santé publique](#toc0_)


![https://www.hospitalia.fr/photo/art/grande/54117300-40801349.jpg?v=1614005985](https://www.hospitalia.fr/photo/art/grande/54117300-40801349.jpg?v=1614005985)


## <a id='toc1_1_'></a>[Présentation du projet](#toc0_)

L'agence Santé publique France souhaite améliorer sa base de données **Open Food Facts**,base de données colaborative, libre et ouverte des produits alimentaire du monde entier. Elle contient notamment les informations de produits alimentaire (ingrédients,allergènes,valeurs nutritionnelles,qualité nutritionnelle (nutriscore)).

- **Objectif du projet** :
  - Nettoyage du jeu de données
  - Analyse exploratoire du jeu de données
    <br>
- **Livrable attendu** :
  1. Un notebook contenant l’ensemble de vos traitements des données ainsi que vos analyses
  2. Un support de présentation


**Table of contents**<a id='toc0_'></a>

- [Projet 3 Préparez des données pour un organisme de santé publique](#toc1_)
  - [Présentation du projet](#toc1_1_)
  - [Importation des libraries python](#toc1_2_)
  - [Importation du jeu de données](#toc1_3_)
  - [Nettoyage des données](#toc1_4_)
    - [Correction des types de variables](#toc1_4_1_)
      - [Conversion des dates](#toc1_4_1_1_)
      - [Conversion geocode](#toc1_4_1_2_)
    - [Remplacement des valeurs manquantes non prises en compte](#toc1_4_2_)
    - [Visualisation des valeurs manquantes](#toc1_4_3_)
      - [Visualisation des valeurs manquantes Graphiques](#toc1_4_3_1_)

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->


## <a id='toc1_2_'></a>[Importation des libraries python](#toc0_)


In [48]:
# Library Import
import pandas as pd

import plotly.express as px
import missingno as msno
import numpy as np
import os
import pandas as pd
import urllib.request
import zipfile
import plotly.graph_objects as go
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
import plotly.graph_objs as go
from plotly.subplots import make_subplots

ModuleNotFoundError: No module named 'plotly'

In [ ]:
# Modification des affichages de colonnes, lignes et largeurs de colonnes pour avoir un maximum d'information
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 1000)

## <a id='toc1_3_'></a>[Importation du jeu de données](#toc0_)


In [ ]:

def import_dataset(dataframe: pd.DataFrame):
    '''
        Import the dataset openfoodfacts.
        This function checks if the specified CSV file exists locally,create a folder called 'dataset' if it doesn't exist,downloads zip file containing the CSV file from remote URL if not found.
        It then imports the data into the provided DataFrame.

    Args :
        dataframe (pd.DataFrame): Give a name to the Pandas DataFrame to store the imported CSV.

    Return :
        dataframe (pd.DataFrame): Return the Pandas DataFrame containing the imported CSV.

    '''

    # Specify the folder path and csv_file_path and too the download URL fir zip file download
    folder_path = 'dataset'
    download_csv = 'https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/parcours-data-scientist/P2/fr.openfoodfacts.org.products.csv.zip'
    csv_file_path = os.path.join(
        folder_path, 'fr.openfoodfacts.org.products.csv')

    # Check if the folder  'dataset' exists
    if not os.path.exists(folder_path):
        # Create the folder if it doesn't exist
        os.makedirs(folder_path)
        print(f"Création du dossier: {folder_path}")
    if not os.path.exists(csv_file_path):
        # Specify the file path for the downloaded zip file
        zip_file_path = os.path.join(
            folder_path, 'fr.openfoodfacts.org.products.csv.zip')

        # Download the zip file
        urllib.request.urlretrieve(download_csv, zip_file_path)
        print(f"Téléchargement du fichier {download_csv}")

        # Uncompress the zip file
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(folder_path)
        print("Extraction du fichier CSV du zip")

        # Specify the file path for the extracted CSV file
        csv_file_path = os.path.join(
            folder_path, 'fr.openfoodfacts.org.products.csv')
        print(f'Lecture du fichier dans le répertoire : {csv_file_path}')
    else:
        print(f'Lecture du fichier dans le répertoire : {csv_file_path}')
    # Read the CSV file using pandas
    # Create a list for specify values that treadted as missing values
    na_values = ["NA", "N/A", "NaN", "", "null", "None", "."]
    dataframe = pd.read_csv(csv_file_path, sep="\t",
                            na_values=na_values, low_memory=False)
    print(f"CSV importé en dataframe via la variable df_food")
    return dataframe

In [ ]:
# Create an empty DataFrame to store the imported data
df_food = pd.DataFrame()

# Call the function and pass the empty DataFrame
df_food = import_dataset(dataframe=df_food)

df_food.head(2)

## <a id='toc1_4_'></a>[Nettoyage des données](#toc0_)

Nous allons débuter notre exploration du jeu de donnée en une connaissance des variables présente et les comprendre pour déterminer les variables à garder pour la suite de notre analyse.<br>
Pour mieux comprendre les champs disponible dans notre base de donnée un lien est [disponible ici](https://world.openfoodfacts.org/data/data-fields.txt).<br>
Grâce a ce document on peut avoir une première catégorisation de nos champs :

- Les informations générales sur la fiche du produit : nom, date de modification, etc.
- Un ensemble de tags : catégorie du produit, localisation, origine, etc.
- Les ingrédients composant les produits et leurs additifs éventuels.
- Des informations nutritionnelles : quantité en grammes d’un nutriment pour 100 grammes du produit.


In [ ]:
# See number of columns and rows on the dataset
print('Nombre de lignes du dataset:',
      df_food.shape[0], '\nNombre de colonnes du dataset:', df_food.shape[1])

In [ ]:
# Create a list with columns dataframe
list(df_food.columns)

### <a id='toc1_4_1_'></a>[Correction des types de variables](#toc0_)

Grâce à la description des colonnes sur le site on peu remarquer que certaines colonne on été convertie dans de mauvais types par pandas.Quand on transforme notre CSV en dataframe on le message suivant :

```python
DtypeWarning: Columns (0,3,5,19,20,24,25,26,27,28,35,36,37,38,39,48) have mixed types. Specify dtype option on import or set low_memory=False.
```

On va spécifier un type pour chaque colonne de notre dataframe


In [ ]:
# See different data types in the column
print('types de variable dans le dataset:\n', df_food.dtypes.value_counts())

In [ ]:
def convert_types(csv_file_path: str):
    """
    Convert data types of columns in a csv file.Need CSV file path to convert types.
    We specify the different new type for each column.\n
    Args :
        csv_file_path (str) : path of the CSV file to convert
    returns :
        dataframe (pd.DataFrame) : Return the Pandas Dataframe with new data type.
        dtypes_columns : dictionary with column name and new data type for enter this variable on read_csv function
        date_list : list with contain column name of column need to convert to datetime (we use this variable in other function).
        specific_dtype : list with contain the column name with specific transform column (conversion geocode/we use this variable in other function).
    """
    #
    dtypes_columns = {}
    date_list = []
    change_type = []
    specific_dtype = []


# loop through the columns for attribute new data type
    for column in (df_food.columns):
        # dtypes object
        if column in ['code', 'url', 'creator', 'product_name', 'generic_name', 'quantity', 'packaging', 'brands', 'origins', 'manufacturing_places', 'manufacturing_places_tags', 'packaging_tags', 'brands_tags', 'categories', 'categories_tags', 'categories_fr', 'origins_tags', 'labels', 'labels_tags', 'labels_fr', 'emb_codes', 'emb_codes_tags', 'cities', 'cities_tags', 'purchase_places', 'stores', 'countries', 'countries_tags', 'countries_fr', 'ingredients_text', 'allergens', 'allergens_fr', 'traces', 'traces_tags', 'traces_fr', 'serving_size', 'additives', 'additives_tags', 'additives_fr', 'ingredients_from_palm_oil_tags', 'ingredients_that_may_be_from_palm_oil_tags', 'states', 'states_tags', 'states_fr', 'main_category', 'main_category_fr', 'image_url', 'image_small_url']:
            old_dtype = df_food[column].dtype
            new_dtype = 'object'
            dtypes_columns[column] = new_dtype
            change_type.append(f"{column}: {old_dtype} --> {new_dtype}")
            # dtypes float
        elif column in ['ingredients_from_palm_oil_n', 'ingredients_from_palm_oil', 'ingredients_that_may_be_from_palm_oil_n', 'ingredients_that_may_be_from_palm_oil', 'no_nutriments', 'additives_n', 'energy_100g', 'energy-from-fat_100g', 'fat_100g', 'saturated-fat_100g', 'butyric-acid_100g', 'caproic-acid_100g', 'caprylic-acid_100g', 'capric-acid_100g', 'lauric-acid_100g', 'myristic-acid_100g', 'palmitic-acid_100g', 'stearic-acid_100g', 'arachidic-acid_100g', 'behenic-acid_100g', 'lignoceric-acid_100g', 'cerotic-acid_100g', 'montanic-acid_100g', 'melissic-acid_100g', 'monounsaturated-fat_100g', 'polyunsaturated-fat_100g', 'omega-3-fat_100g', 'alpha-linolenic-acid_100g', 'eicosapentaenoic-acid_100g', 'docosahexaenoic-acid_100g', 'omega-6-fat_100g', 'linoleic-acid_100g', 'arachidonic-acid_100g', 'gamma-linolenic-acid_100g', 'dihomo-gamma-linolenic-acid_100g', 'omega-9-fat_100g', 'oleic-acid_100g', 'elaidic-acid_100g', 'gondoic-acid_100g', 'mead-acid_100g', 'erucic-acid_100g', 'nervonic-acid_100g', 'trans-fat_100g', 'cholesterol_100g', 'carbohydrates_100g', 'sugars_100g',
                        'sucrose_100g', 'glucose_100g', 'fructose_100g', 'lactose_100g', 'maltose_100g', 'maltodextrins_100g', 'starch_100g', 'polyols_100g', 'fiber_100g', 'proteins_100g', 'casein_100g', 'serum-proteins_100g', 'nucleotides_100g', 'salt_100g', 'sodium_100g', 'alcohol_100g', 'vitamin-a_100g', 'beta-carotene_100g', 'vitamin-d_100g', 'vitamin-e_100g', 'vitamin-k_100g', 'vitamin-c_100g', 'vitamin-b1_100g', 'vitamin-b2_100g', 'vitamin-pp_100g', 'vitamin-b6_100g', 'vitamin-b9_100g', 'folates_100g', 'vitamin-b12_100g', 'biotin_100g', 'pantothenic-acid_100g', 'silica_100g', 'bicarbonate_100g', 'potassium_100g', 'chloride_100g', 'calcium_100g', 'phosphorus_100g', 'iron_100g', 'magnesium_100g', 'zinc_100g', 'copper_100g', 'manganese_100g', 'fluoride_100g', 'selenium_100g', 'chromium_100g', 'molybdenum_100g', 'iodine_100g', 'caffeine_100g', 'taurine_100g', 'ph_100g', 'fruits-vegetables-nuts_100g', 'collagen-meat-protein-ratio_100g', 'cocoa_100g', 'chlorophyl_100g', 'carbon-footprint_100g', 'nutrition-score-fr_100g', 'nutrition-score-uk_100g', 'glycemic-index_100g', 'water-hardness_100g']:
            old_dtype = df_food[column].dtype
            new_dtype = 'float'
            dtypes_columns[column] = new_dtype
            change_type.append(f"{column}: {old_dtype} --> {new_dtype}")
            # dtypes category
        elif column in ['nutrition_grade_uk', 'nutrition_grade_fr', 'pnns_groups_1', 'pnns_groups_2']:
            old_dtype = df_food[column].dtype
            new_dtype = 'category'
            dtypes_columns[column] = new_dtype
            change_type.append(f"{column}: {old_dtype} --> {new_dtype}")
            # dtypes int
        elif column in []:
            old_dtype = df_food[column].dtype
            new_dtype = 'int'
            dtypes_columns[column] = new_dtype
            change_type.append(f"{column}: {old_dtype} --> {new_dtype}")
            # dtypes datetime
        elif column in ['created_t', 'created_datetime', 'last_modified_t', 'last_modified_datetime']:
            old_dtype = df_food[column].dtype
            new_dtype = 'datetime'
            change_type.append(f"{column}: {old_dtype} --> {new_dtype}")
            date_list.append(column)
            # specific dtype
        else:
            specific_dtype.append(column)
    # Apply read csv function with the new dtype conversion
    df = pd.read_csv("dataset/fr.openfoodfacts.org.products.csv",
                     dtype=dtypes_columns, low_memory=False, sep="\t")
    # Print changing types for each column with seperate lines
    print("\n".join(change_type))
    return df, date_list, specific_dtype

In [ ]:
# Call the function to convert types, store date list and specific dtype
df_food, date_list, specific_dtype = convert_types(
    "dataset/fr.openfoodfacts.org.products.csv")

#### <a id='toc1_4_1_1_'></a>[Conversion des dates](#toc0_)

- Les dates qui termine \_t sont des dates en UNIX timestamp format
- Les dates qui termine \_datetime sont des dates in the iso8601 format : yyyy-mm-ddThh:mn:ssZ


In [ ]:
france_rows = df_food[df_food['created_t'] == 'France']

display(france_rows)

In [ ]:
def convert_date(date_list: list, df: pd.DataFrame):
    """
    Conversion of date in datetime with specific format 

    Args :
        date_list : list of date from columns dataframe
        df : dataframe to convert
    Returns :
        dataframe with converted date
    """
    for column in date_list:
        if column.endswith("_t"):
            # Format as Unix timestamp (seconds since epoch)
            print(column)
            df[column] = pd.to_datetime(df[column], unit='s', errors='coerce')
        elif column.endswith("_datetime"):
            # Format as ISO 8601 datetime
            df[column] = pd.to_datetime(
                df[column], format='%Y-%m-%dT%H:%M:%SZ', errors='coerce')
    # Display dataframe with only datetime columns
    display(df[['created_t', 'created_datetime',
            'last_modified_t', 'last_modified_datetime']])
    return df

In [ ]:
# Use the function convert_date
df_food = convert_date(date_list, df_food)

#### <a id='toc1_4_1_2_'></a>[Conversion geocode](#toc0_)

La colonne 'first_packaging_code_geo' nous donne les cordonnées des packagings des produits.
Nous allons supprimer cette colonne pour ajouter deux nouvelle colonne ayant la latitude et la longitude.


In [ ]:
display(df_food[['first_packaging_code_geo']].value_counts())

In [ ]:
def convert_geocode(specific_dtype: list, df: pd.DataFrame):
    """
    Create new columns for latitude and longitude from first_packaging_code_geo column
    """
    for column in specific_dtype:
        # Split the first_packaging_code_geo column into two columns
        df[['fp_lat', 'fp_lon']] = df[column].str.split(
            ',', n=1, expand=True)
        # Convert fp_lat and fp_lon to float
        df.fp_lat = round(df.fp_lat.astype(float), 2)
        df.fp_lon = round(df.fp_lon.astype(float), 2)
        # drop first_packaging_code_geo column
        df = df.drop(columns="first_packaging_code_geo")
        display(df[['fp_lat', 'fp_lon']])
    return df

In [ ]:
df_food = convert_geocode(specific_dtype, df_food)

### <a id='toc1_4_2_'></a>[Remplacement des valeurs manquantes non prises en compte](#toc0_)

Il existe un grand nombre de valeur manquante dans notre dataframe nous allons vérifier si il n'y a pas d'autres valeurs manquantes. On sait que les valeurs manquantes sont comptabiliser par pandas avant les valeurs suivantes :NaN, None, or NaT. Vérifions maintenant qu'il n'existe pas d'autres valeurs manquantes et remplaçons les.


In [ ]:
# Create a function to replace missing values
def replace_missing_values(dataframe: pd.DataFrame, missing_values=['N/A', '-', 'Nan', 'nan']):
    missing_count = dataframe.isna().sum().sum()
    print(f"Number of Missing Values: {missing_count}")
    for col in dataframe.columns:
        dataframe[col] = dataframe[col].replace(missing_values, np.nan)
    print(f"Number of Missing Values: {missing_count}")
    return dataframe, missing_count

In [ ]:
df, missing_count = replace_missing_values(
    df_food, missing_values=['N/A', '-', 'Nan', 'nan'])

### Suppression des valeurs dupliqués

Avec les informations que l'on dispose on sait de la colonne code représente le code de chaque produit et doit donc être unique pour chaque produit dans notre dataframe. Vérifions si il existe des produits doublonnés.


In [ ]:
# Verify if
# Choose the column for which you want to count unique values
selected_column = 'code'

# Get the number of unique values in the selected column
unique_values = df_food[selected_column].nunique()

# Get the total number of values in the selected column
total_values = len(df_food[selected_column])

# Calculate the difference
difference = total_values - unique_values
# Print unique/non unique values in the column
print(f"Nombre de valeurs uniques dans '{selected_column}': {unique_values}")
print(f"Nombre de valeurs égales: {difference}")
# Display non-unique values in the 'code' column
non_unique_values = df_food[df_food.duplicated(
    subset=[selected_column], keep=False)]
print(f"valeur non-unique dans la colonne '{selected_column}':")
print(non_unique_values)

In [ ]:
def drop_missing_values_column(dataframe: pd.DataFrame, column_name: str):
    """
    Drop rows with missing values in a specific column of a DataFrame.

    Parameters:
    dataframe (DataFrame): The input DataFrame.
    column_name (str): The name of the column in which to drop missing values.

    Returns:
    DataFrame: A new DataFrame with rows containing missing values in the specified column removed.
    int: The number of rows before dropping missing values.
    int: The number of rows after dropping missing values.
    """
    # Store the number of rows before dropping missing values
    original_rows = len(dataframe)

    # Create a copy of the DataFrame
    df_copy = dataframe.copy()

    # Drop rows with missing values in the specified column
    df_copy.dropna(subset=[column_name], inplace=True)

    # Store the number of rows after dropping missing values
    modified_rows = len(df_copy)
    # Check if rows have been delete in the dataframe
    print(
        f"Nombre de ligne dans le DataFrame avant suppression: {original_rows}")
    print(
        f"Nombre de ligne dans le DataFrame après suppression: {modified_rows}")
    return df_copy, original_rows, modified_rows

In [ ]:
df_food, original_rows, modified_rows = drop_missing_values_column(
    df_food, 'code')

Nous avons pu vérifier qu'il n'existe que des valeurs unique dans cette colonne, nous avons simplement supprimer les valeurs manquantes dans cette colonne


### <a id='toc1_4_3_'></a>[Visualisation des valeurs manquantes](#toc0_)


Regardons ensemble le taux de complétion de notre dataframe par colonnes


In [ ]:
# Create a function to see the completion of dataframe per column
def missing_value_dataframe(dataframe: pd.DataFrame):
    """Function for calculating missing values per column to dataframe. 
    Get the percentage of non-missing and total missing values per column

    parameters :
        dataframe : pandas dataframe to calculate missing values
    return :
    Specific dataframe with missing values per column   
    """
    # Count missing values per column
    missing_count = dataframe.isnull().sum()

    # Count non-missing values per column
    non_missing_count = dataframe.notnull().sum()

    # Calculate the total number of values per column
    total_count = len(dataframe)

    # Calculate the percentage of non-missing values
    percentage_non_missing = (non_missing_count / total_count) * 100

    # Create a new DataFrame with the values
    df_missing_values = pd.DataFrame({
        'Taux de remplissage': percentage_non_missing,
        'Nombre de valeurs manquantes': missing_count})
    df_missing_values = df_missing_values.sort_values(
        by='Taux de remplissage', ascending=False)

    return df_missing_values

In [2]:
df_missing_values = missing_value_dataframe(df_food)
display(df_missing_values)

NameError: name 'missing_value_dataframe' is not defined

#### <a id='toc1_4_3_1_'></a>[Visualisation des valeurs manquantes Graphiques](#toc0_)


In [3]:
# Use missingno to visualize missing data with graphic
msno.bar(df_food, sort="ascending", color="dodgerblue")

NameError: name 'msno' is not defined

In [4]:
# Calculate the count of missing values and non-missing values
missing_count = df_food.isna().sum().sum()
non_missing_count = df_food.count().sum()
# Create a pie chart for see the representation of dataframe
fig = px.pie(
    names=['Valeurs manquantes', 'Valeurs non-manquantes'],
    values=[missing_count, non_missing_count],
    title="Représentation des valeurs manquantes/non manquantes dataframe",
)
# Show the pie chart
fig.show()

NameError: name 'df_food' is not defined

In [5]:
# Create a chart pie for group column in different pourcentage

def categorize_percentage(percentage):
    """
    """
    if percentage == 100:
        return "100%"
    elif percentage < 100 and percentage >= 90:
        return "99-90%"
    elif percentage >= 50 and percentage < 90:
        return "89-50%"
    elif percentage <= 50 and percentage >= 10:
        return "50-10%"
    elif percentage <= 10 and percentage >= 1:
        return "10-01%"
    elif percentage <= 1 and percentage > 0.0:
        return "1-0.01%"
    else:
        return "0%"


# Apply the categorize_percentage function to the DataFrame
df_missing_values['Pourcentage_category'] = df_missing_values['Taux de remplissage'].apply(
    categorize_percentage)

# Group and count the percentages by category
category_counts = df_missing_values['Pourcentage_category'].value_counts(
).reset_index()
category_counts.columns = ['Pourcentage_category', 'Count']

# Create a pie chart with the grouped percentages
fig = px.pie(category_counts, names='Pourcentage_category',
             values='Count', title='Nombre de colonne par pourcentage de complétion')
# Add a title to the legend
fig.update_layout(legend_title_text='Groupement des colonnes par pourcentage')
# Show the pie chart
fig.show()

NameError: name 'df_missing_values' is not defined

#### Obervation des valeurs manquantes dans notre dataframe

- **76 % de notre dataframe contient des valeurs manquantes**
- **3% de notre dataframe ne contient aucune valeurs manquantes**, soit 5/162 colonnes
- **10% de notre dataframe a taux de complétion compris entre 100% et 90%**, soit 16/162 colonnes
- **11% de notre dataframe a taux de complétion compris entre 89% et 50%**, soit 18/162 colonnes
- **79% de notre dataframe a taux de complétion inférieur a 50%**, soit 130/162 colonnes
- **13% de notre dataframe ne contient que des valeurs nulle**, soit 21/162 colonnes


Maintenant que nous avons d'information sur les valeurs manquantes dans notre dataframe, nous allons sélectionner les colonnes en fonction de leurs pertinance pour notre analyse ainsi qu'en fonction de la complétion dans le dataframe


### Méthode de calcul du Nutri-Score

On sait que la valeurs la plus importantes dans notre jeux de données est le _Nutri-Score_.
Mais comment cette valeurs est t-elle calculé ?


#### Quesce-ce que le Nutri-Score ?

"C'est une note permettant d'avoir une information sur la qualité nutritionnelle globale d'un produit alimentaire. Son but est d'aider les consommateurs au choix d'aliments sains pour sa santé.

Elle se base sur les travaux de l’équipe du Professeur Serge Hercberg (Université Paris 13), les expertises de l’Anses et du Haut Conseil de Santé Publique."[(Source)]('https://simonetthomas.github.io/CalculateurNutriscore/faq.html#nutriscore')


#### Comment est-il calculé ?

![nutriscore_image](https://assets-global.website-files.com/5e4d08c15370e9073c634a54/62e3febf1c959db9cb85c2a6_le-nutri-score-debarque-chez-foodles-calcul-food-in-action.jpeg)

"La méthode de calcul est basée sur l'assignation de points en fonction de la valeur de chaque nutriment considéré comme « à éviter » ou « à favoriser ». Les valeurs sur lesquelles on se base sont celles indiquées dans la « déclaration nutritionnelle obligatoire », souvent présent sous la forme d'un tableau à l'arrière du produit.

Les éléments à éviter sont ceux dont les points sont indiqués en rouge dans le tableau : apport calorique, acides gras saturés, sucres, et sodium (calculé d'après le sel). Les éléments à favoriser sont indiqués en vert : teneur en fibres, protéines, et en fruits, légumes et noix.

La règle générale est de soustraire ensuite ces deux sous-totaux de points (points négatifs - points positifs), résultant en un score nutritionnel, le plus faible étant le meilleur. Une note de A à E est ensuite déterminée sur la base de ce score, A étant la meilleure, et E la plus mauvaise."[(Source)]('https://simonetthomas.github.io/CalculateurNutriscore/faq.html#nutriscore')


- Observation :
- Dans le calcul du nutri-score voici les informations nutritionnelles qui sont pris en compte :
  - Energie (kJ)
  - Acides gras saturés (g)
  - Sucres (g)
  - Fibres (g)
  - Protéines (g)
  - Sel (g)
  - Sodium (g)
  - Fruits,légumes et noix (%) calcul des ingrédients représentant des fruits,légumes, ou noix dans la liste d'ingrédients


### Filtrage de notre dataframe


- Choix des colonnes :

  - Information Nutritionnel présent dans le calcul du nutri-score ('energy_100g', 'proteins_100g', 'sugars_100g', 'fiber_100g', 'salt_100g'). Non garderons pas le 'sodium' car c'est un ingrédients pour le sel.
  - Informations générale sur le produit ('code', 'url', 'product_name', 'brands', 'countries_fr')
  - Information annexes sur le produit ('additives_fr', 'quantity', 'pnns_groups_1', 'pnns_groups_2', 'ingredients_text')


In [6]:
def filter_dataframe(dataframe, columns_to_select):
    """
    Select specific columns from a DataFrame and create a new DataFrame with those columns.

    Parameters:
    - dataframe: The original DataFrame.
    - columns_to_select: A list of column names to select.

    Returns:
    - A new DataFrame containing only the selected columns.
    """
    if not set(columns_to_select).issubset(dataframe.columns):
        # Check if all columns_to_select are present in the original DataFrame
        missing_columns = set(columns_to_select) - set(dataframe.columns)
        raise ValueError(
            f"Columns not found in the DataFrame: {missing_columns}")

    dataframe = dataframe[columns_to_select].copy()
    # Conversion factor from kJ to kcal
    conversion_factor = 0.239006

    # Create a new column 'energy_100g_kcal' with energy values in kcal
    dataframe['energy_100g_kcal'] = dataframe['energy_100g'] * \
        conversion_factor
    # Drop the original 'energy_100g' column to keep energy_100g_kcal
    dataframe.drop(columns=['energy_100g'], inplace=True)

    return dataframe

In [7]:
# Use function filter_dataframe for select columns in dataframe **df_food**

df_filtered = filter_dataframe(df_food, ['code', 'url', 'product_name', 'brands', 'countries_fr', 'nutrition-score-fr_100g', 'nutrition_grade_fr',
                               'additives_n', 'additives_fr', 'quantity', 'energy_100g', 'proteins_100g', 'sugars_100g', 'fiber_100g', 'salt_100g'])

NameError: name 'df_food' is not defined

#### Nettoyage des données filtrés


##### Nettoyage de la colonne **countries_fr**


In [8]:
df_filtered['countries_fr'].to_list()

NameError: name 'df_filtered' is not defined

In [9]:

def drop_rows_not_containing_substring(dataframe, column_name, substring):
    # Create a boolean mask to identify rows where the specified column does not contain the substring
    mask = dataframe[column_name].str.contains(substring, case=False, na=False)

    # Use the mask to filter the DataFrame, keeping only the rows that match the condition
    dataframe = dataframe[mask]
    # Get the number of rows in each dataframe
    number_rows_old_df = len(df_food)
    number_rows_new_df = len(dataframe)
    difference_num_rows = number_rows_old_df - number_rows_new_df

    # Print the number of rows
    print(
        f"Nombre de lignes dans le dataframe n'ayant pas le mot france: {difference_num_rows}")
    print(
        f'Nombre de lignes dans le dataframe contenant uniquement des produits vendu en France: {number_rows_new_df}')

    return dataframe


# Call the function to drop rows not containing 'France'
df_filtered = drop_rows_not_containing_substring(
    df_filtered, 'countries_fr', 'France')

NameError: name 'df_filtered' is not defined

Maintenant que notre dataset contient uniquement des données de produit vendues en France,regardons les données manquantes dans nos données filtrées.

In [10]:
missing_value_dataframe(df_filtered)

NameError: name 'missing_value_dataframe' is not defined

In [11]:

# Calculate the percentage of rows with missing values in both columns A and B
percentage_missing_values_both = (df_filtered['product_name'].isnull(
) & df_filtered['brands'].isnull()).sum() / len(df_filtered) * 100

# Display the result
print(
    f"Percentage of rows with missing values in both columns product_name and brands: {percentage_missing_values_both:.2f} %")

NameError: name 'df_filtered' is not defined

In [12]:
def drop_missing_values(dataframe, columns_to_drop):
    """
    Drops rows with missing values in specified columns from the DataFrame.

    Parameters:
    - df (pd.DataFrame): The input DataFrame.
    - columns_to_drop (list): List of column names where missing values should be dropped.

    Returns:
    - pd.DataFrame: .
    """
    df_dropped = dataframe.dropna(subset=columns_to_drop)
    dataframe = missing_value_dataframe(df_dropped)
    print(
        f'Suppression des valeurs manquantes dans les colonnes {columns_to_drop}')
    return dataframe

In [13]:
# Call the function for drop columns with missing values
drop_missing_values(df_filtered, ['product_name', 'brands'])

NameError: name 'df_filtered' is not defined

#### Traitement des données manquantes


#### Les types de données manquantes

Avant le traitement des données manquantes par différentes méthodes d'imputation,nous devons identifier que type de données manquantes nous avons dans nos échantillons de données.
Il existe différent types de données manquantes :

- **MCAR** (Missing Completely at Random) Une donnée est MCAR, c’est-à-dire manquante de façon complètement aléatoire si la probabilité d’absence est la même pour toutes les observations. Cette probabilité ne dépend donc que de paramètres extérieurs indépendants de cette variable
- **MAR** (Missing At Random) les valeurs manquantes dépendent des autres données observées, mais pas des données manquantes
- **MNAR** (Missing Not At Random) La donnée est manquante de façon non aléatoire si la probabilité d’absence dépend de la variable en question


##### Visualisation des données manquantes de nos données filtrés


In [14]:
missing_value_dataframe(df_filtered)

NameError: name 'missing_value_dataframe' is not defined

#### Identification des types de données manquantes


In [15]:
msno.heatmap(df_filtered, labels=True)

NameError: name 'msno' is not defined

In [16]:
msno.dendrogram(df_filtered)

NameError: name 'msno' is not defined

Grâce aux visualisation graphique de nos données manquantes, on peut remarquer que nos données sont fortement liées aux autre variables manquantes, on a donc dans notre jeux de données filtrer des données manquantes de types **MAR**. Nous allons Imputer nos données via la méthode **KNN Inputer** pour nos données numérique et pour nos données catégorielle nous allons utiliser le **SimpleImputer** 


##### KNN Imputer 


In [17]:
# Skip this cell when running notebook
%%skip

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))


def optimize_k(data, target, categorical_cols):
    errors = []
    for k in range(1, 30):

        imputer = KNNImputer(n_neighbors=k)
        imputed_data = imputer.fit_transform(data)
        df_imputed = pd.DataFrame(imputed_data, columns=data.columns)

        X = df_imputed.drop(target, axis=1)
        y = df_imputed[target]
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42)

        model = RandomForestRegressor()
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        error = rmse(y_test, preds)
        errors.append({'K': k, 'RMSE': error})

    return errors


# Identify numerical and categorical columns
numerical_cols = df_filtered.select_dtypes(include=[np.number])
categorical_cols = df_filtered.select_dtypes(exclude=[np.number])

# Specify the target column
target_column = 'nutrition-score-fr_100g'


# Call the optimization function
k_errors = optimize_k(data=numerical_cols,
                      target=target_column, categorical_cols=categorical_cols)

# Print or analyze the results
print(k_errors)

UsageError: Line magic function `%%skip` not found.


In [18]:
k_rmse = [{'K': 1, 'RMSE': 2.584246879542272}, {'K': 2, 'RMSE': 2.592634459053153}, {'K': 3, 'RMSE': 2.6557456621641498}, {'K': 4, 'RMSE': 2.7157149608094495}, {'K': 5, 'RMSE': 2.6827404859081887}, {'K': 6, 'RMSE': 2.651040719334745}, {'K': 7, 'RMSE': 2.6679174745720777}, {'K': 8, 'RMSE': 2.6536641153484988}, {'K': 9, 'RMSE': 2.654174432555565}, {'K': 10, 'RMSE': 2.632810139669687}, {'K': 11, 'RMSE': 2.6053500206147437}, {'K': 12, 'RMSE': 2.6093688853118984}, {'K': 13, 'RMSE': 2.589366016342716}, {'K': 14, 'RMSE': 2.5959565924885717}, {
    'K': 15, 'RMSE': 2.591174671693854}, {'K': 16, 'RMSE': 2.5683151864038667}, {'K': 17, 'RMSE': 2.55483794464236}, {'K': 18, 'RMSE': 2.5388365709891048}, {'K': 19, 'RMSE': 2.527759984443189}, {'K': 20, 'RMSE': 2.5261167705241867}, {'K': 21, 'RMSE': 2.522365320439682}, {'K': 22, 'RMSE': 2.5339303756037523}, {'K': 23, 'RMSE': 2.5134252907793577}, {'K': 24, 'RMSE': 2.5031421089065082}, {'K': 25, 'RMSE': 2.520884348637867}, {'K': 26, 'RMSE': 2.5120377513343772}, {'K': 27, 'RMSE': 2.506308660617449}, {'K': 28, 'RMSE': 2.4953048476194835}, {'K': 29, 'RMSE': 2.485284666373997}]

results = pd.DataFrame(data=k_rmse, columns=['K', 'RMSE'])
fig = go.Figure()
fig.add_trace(go.Scatter(x=results['K'], y=results['RMSE']))
fig.update_layout(title='RMSE pour les différentes valeurs de K',
                  xaxis_title='k', yaxis_title='Root Mean Squared Error')

NameError: name 'go' is not defined

In [19]:

columns_with_missing_values = df_filtered.columns[df_filtered.isnull(
).any()].tolist()

df_missing_values = df_filtered[columns_with_missing_values]

# Identify numerical and categorical columns
numerical_cols = df_missing_values.select_dtypes(include=[np.number]).columns
categorical_cols = df_missing_values.select_dtypes(exclude=[np.number]).columns

# Impute numerical columns using KNNImputer
imputer = KNNImputer(n_neighbors=29)
df_numerical_imputed = pd.DataFrame(imputer.fit_transform(
    df_missing_values[numerical_cols]), columns=numerical_cols)

# Impute categorical columns using SimpleImputer with 'most_frequent' strategy
categorical_imputer = SimpleImputer(strategy='most_frequent')

df_categorical_imputed = pd.DataFrame(categorical_imputer.fit_transform(
    df_missing_values[categorical_cols]), columns=categorical_cols)


# Concatenate the results
df_imputed = pd.concat([df_numerical_imputed, df_categorical_imputed], axis=1)

print("Imputed DataFrame:")
print(df_imputed)

NameError: name 'df_filtered' is not defined

In [20]:
missing_value_dataframe(df_imputed)

NameError: name 'missing_value_dataframe' is not defined

#### Détection des outliers


##### Comparaison de nos données filtrées et données imputées

In [21]:
df_filtered.describe()

NameError: name 'df_filtered' is not defined

In [22]:
df_imputed.describe()

NameError: name 'df_imputed' is not defined

##### Idenfication graphique des outliers

In [23]:
# Select numeric value from df_imputed
df_outlier_graph_nutri = df_imputed[['proteins_100g',
                                     'sugars_100g', 'fiber_100g', 'salt_100g', 'nutrition-score-fr_100g', 'additives_n']]
fig = make_subplots(rows=1, cols=len(
    df_outlier_graph_nutri.columns), shared_yaxes=True)

# Iterate through each column and create a violin plot
for i, col in enumerate(df_outlier_graph_nutri.columns, start=1):
    trace = go.Violin(y=df_outlier_graph_nutri[col], name=col)
    fig.add_trace(trace, row=1, col=i)

# Update layout
fig.update_layout(
    title_text="Distribution des informations nutritionnelles", showlegend=False)
fig.update_yaxes(title_text="Valeurs de chaque colonne", row=1, col=1)

# Show the plot
fig.show()

NameError: name 'df_imputed' is not defined

#### Identification des outliers avec IQR


In [24]:
# Function for detect outliers with IQR Method
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    data.index[(data[column] < lower_bound) | (
        data[column] > upper_bound)]
    return data.index[(data[column] < lower_bound) | (
        data[column] > upper_bound)]


# Create an empty list to store the output indices from multiple columns
index_list = []
for feature in ['nutrition-score-fr_100g', 'energy_100g_kcal', 'proteins_100g', 'sugars_100g', 'fiber_100g', 'salt_100g', 'additives_n']:
    # use the function
    index_list.extend(detect_outliers_iqr(df_imputed, feature))
    print(f"Outliers détecter pour la colonne {feature}: {len(index_list)}")
    # See rows with outliers

display(df_imputed.iloc[index_list])

NameError: name 'df_imputed' is not defined

In [25]:
def detect_outliers_iqr(data, features):
    outliers = pd.DataFrame(index=data.index)

    for feature in features:
        Q1 = data[feature].quantile(0.25)
        Q3 = data[feature].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Create a mask for outliers for the current feature
        outlier_mask = (data[feature] < lower_bound) | (
            data[feature] > upper_bound)

        # Add a new column indicating whether the feature is an outlier
        outliers[feature + '_outlier'] = outlier_mask.astype(int)

    # Sum the outlier columns to count the total number of outlier features for each row
    outliers['total_outliers'] = outliers.sum(axis=1)

    return outliers[outliers['total_outliers'] > 0]


# Example usage:
outliers_info = detect_outliers_iqr(df_imputed, [
                                    'nutrition-score-fr_100g', 'energy_100g_kcal', 'proteins_100g', 'sugars_100g', 'fiber_100g', 'salt_100g', 'additives_n'])

# Display the DataFrame with information about outlier features
display(outliers_info)

NameError: name 'df_imputed' is not defined

##### Représentation graphique des outliers avec la méthodes IQR pour chaque colonnes

In [26]:
import pandas as pd
import plotly.express as px

# Function for detecting outliers with IQR Method


def detect_outliers_iqr(data, features):
    outliers = pd.DataFrame(index=data.index)

    for feature in features:
        Q1 = data[feature].quantile(0.25)
        Q3 = data[feature].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Create a mask for outliers for the current feature
        outlier_mask = (data[feature] < lower_bound) | (
            data[feature] > upper_bound)

        # Add a new column indicating whether the feature is an outlier
        outliers[feature + '_outlier'] = outlier_mask.astype(int)

    # Sum the outlier columns to count the total number of outlier features for each row
    outliers['total_outliers'] = outliers.sum(axis=1)

    return outliers[outliers['total_outliers'] > 0]


# Example usage:
# Assuming df_imputed is your DataFrame
features_to_check = ['nutrition-score-fr_100g', 'energy_100g_kcal',
                     'proteins_100g', 'sugars_100g', 'fiber_100g', 'salt_100g', 'additives_n']
outliers_info = detect_outliers_iqr(df_imputed, features_to_check)

# Concatenate outliers_info with df_imputed
df_with_outliers_info = pd.concat([df_imputed, outliers_info], axis=1)
df_with_outliers_info = df_with_outliers_info.fillna(0)

# Create visualizations for each feature
for feature in features_to_check:
    fig = px.scatter(df_with_outliers_info, x=feature, color=feature+'_outlier',
                     labels={feature+'_outlier': f'Outlier ({feature})'},
                     title=f'Outliers Visualization for {feature}')
    fig.update_layout(title_text=f'Outliers Visualization for {feature}')
    fig.show()

ModuleNotFoundError: No module named 'plotly'

#### Idenfication des outliers avec le Z-scores


In [27]:
# Function for detect outliers with Z-score Method
def detect_outliers_zscore(data, column, threshold=2):
    z_scores = (data[column] - data[column].mean()) / data[column].std()
    return data[np.abs(z_scores) > threshold]


# Create an empty list to store the output indices from multiple columns
index_list = []
# Calculate the total number of rows in the DataFrame
total_rows = len(df_imputed)

for feature in ['nutrition-score-fr_100g', 'energy_100g_kcal', 'proteins_100g', 'sugars_100g', 'fiber_100g', 'salt_100g']:
    # use the function
    index_list.extend(detect_outliers_zscore(df_imputed, feature))
    # Calculate the percentage of outliers detected
    percentage_detected = len(index_list) / total_rows * 100

    print(
        f"Outliers détecter dans la colonne {feature}: {len(index_list)}, Pourcentage: {percentage_detected:.2f}%")

print(
    f"Cumulative percentage of outliers detected: {percentage_detected:.2f}%")
# See rows with outliers
# df_outliers = (df_imputed.iloc[index_list])

NameError: name 'df_imputed' is not defined

In [28]:
# Function for detecting outliers with Z-score Method
def detect_outliers_zscore(data, features):

    outliers = pd.DataFrame(index=data.index)

    for feature in features:
        z_scores = (data[feature] - data[feature].mean()) / data[feature].std()

        # Define a z-score threshold (e.g., 3 for a significant outlier)
        z_score_threshold = 3

        # Create a mask for outliers for the current feature
        outlier_mask = (z_scores.abs() > z_score_threshold)

        # Add a new column indicating whether the feature is an outlier
        outliers[feature + '_outlier'] = outlier_mask.astype(int)

    # Sum the outlier columns to count the total number of outlier features for each row
    outliers['total_outliers'] = outliers.sum(axis=1)

    # Get the number of outliers
    num_outliers = len(outliers[outliers['total_outliers'] > 0])

    # Calculate the percentage of outliers
    percentage_outliers = (num_outliers / len(data)) * 100

    return num_outliers, percentage_outliers, outliers


# Example usage for Z-score outlier detection:
features_to_check = ['nutrition-score-fr_100g', 'energy_100g_kcal',
                     'proteins_100g', 'sugars_100g', 'fiber_100g', 'salt_100g', 'additives_n']
num_outliers_zscore, percentage_outliers_zscore, df_outlier_z_score = detect_outliers_zscore(
    df_imputed, features_to_check)

print(f"Number of outliers detected: {num_outliers_zscore}")
print(f"Percentage of outliers detected: {percentage_outliers_zscore:.2f}%")

# Display DataFrame with outlier information
display(df_outlier_z_score)

NameError: name 'df_imputed' is not defined

In [29]:
# Concatenate df_imputed and df_outlier_z_score along columns
combined_z_score = pd.concat([df_imputed, df_outlier_z_score], axis=1)

# Display the combined DataFrame
display(combined_z_score)

NameError: name 'df_imputed' is not defined

#### Isolation Forest


In [30]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

# Function for detecting outliers with Isolation Forest


def detect_outliers_isolation_forest(data, features):
    # Select the features for outlier detection
    X = data[features].values

    # Standardize the data
    scaler = StandardScaler()
    X_standardized = scaler.fit_transform(X)

    # Fit the Isolation Forest model
    clf = IsolationForest(contamination='auto', random_state=42)
    y_pred = clf.fit_predict(X_standardized)

    # Create a DataFrame to store the outlier information
    outliers = pd.DataFrame(index=data.index)
    outliers['is_outlier'] = (y_pred == -1).astype(int)

    # Get the number of outliers
    num_outliers = len(outliers[outliers['is_outlier'] == 1])

    # Calculate the percentage of outliers
    percentage_outliers = (num_outliers / len(data)) * 100

    return num_outliers, percentage_outliers, outliers


# Example usage for Isolation Forest outlier detection:
features_to_check = ['nutrition-score-fr_100g', 'energy_100g_kcal',
                     'proteins_100g', 'sugars_100g', 'fiber_100g', 'salt_100g', 'additives_n']
num_outliers_iforest, percentage_outliers_iforest, df_outlier_iforest = detect_outliers_isolation_forest(
    df_imputed, features_to_check)

print(
    f"Number of outliers detected (Isolation Forest): {num_outliers_iforest}")
print(
    f"Percentage of outliers detected (Isolation Forest): {percentage_outliers_iforest:.2f}%")

# Display DataFrame with outlier information
display(df_outlier_iforest)

NameError: name 'df_imputed' is not defined

In [ ]:
import plotly.graph_objects as go

# Extract the outlier column from df_outlier_iforest
outlier_column = 'is_outlier'

# Create a scatter plot for each feature with outliers highlighted
fig = go.Figure()

for feature in features_to_check:
    fig.add_trace(go.Scatter(
        x=df_imputed.index,
        y=df_imputed[feature],
        mode='markers',
        marker=dict(
            color=np.where(
                df_outlier_iforest[outlier_column] == 1, 'red', 'blue'),
            size=8,
            line=dict(color='black', width=1),
        ),
        name=feature,
    ))

fig.update_layout(
    title='Outliers Detection with Isolation Forest',
    xaxis=dict(title='Data Points'),
    yaxis=dict(title='Feature Values'),
)

fig.show()

In [ ]:
from sklearn.ensemble import IsolationForest

# Assuming df_imputed is your DataFrame
df_outlier_forest = df_imputed[['nutrition-score-fr_100g', 'energy_100g_kcal',
                                'proteins_100g', 'sugars_100g', 'fiber_100g', 'salt_100g', 'additives_n']]

# Apply Isolation Forest
iso_forest = IsolationForest(contamination=0.1, random_state=42)
outlier_labels = iso_forest.fit_predict(df_outlier_forest)

# Add the outlier labels to the DataFrame
df_outlier_forest['Outlier'] = outlier_labels

# Display the outliers
outliers = df_outlier_forest[df_outlier_forest['Outlier'] == -1]

# Calculate the proportion of outliers using .loc to avoid SettingWithCopyWarning
df_outliers = df_outlier_forest.loc[df_outlier_forest['Outlier'] == -1]
proportion_of_outliers = len(df_outliers) / len(df_outlier_forest)

print(f"Pourcentage d'outlier détecter: {proportion_of_outliers:.2%}")

In [ ]:
def visualize_outliers(df, features, outlier_column='Outlier'):
    # Create a scatter plot for each pair of features, colored by the outlier status
    fig = px.scatter_matrix(df, dimensions=features, color=outlier_column)

    # Update layout
    fig.update_layout(title_text='Outliers Visualization using Isolation Forest',
                      height=800)

    # Show the plot
    fig.show()


# Example usage
features_to_visualize = ['nutrition-score-fr_100g', 'energy_100g_kcal',
                         'proteins_100g', 'sugars_100g', 'fiber_100g', 'salt_100g']

visualize_outliers(df_outlier_forest, features_to_visualize)

#### Changement des noms des colonnes


Ce dataset est un dataset pour analyser les produit français, nous allons donc mettre le nom des colonnes en français.


In [ ]:

# def clean_column_names(dataframe):
#     """Function for set name of column in french

#     Args:
#         dataframe ([type]): [description]
#     Returns:
#         dataframe with the modification of columns name
#     """

#    # Rename specific columns if needed

#     column_name_mapping = {
#         'product_name': 'nom_produit',
#         'brands': 'marque',
#         'countries_fr': 'pays',
#         'nutrition_grade_fr': 'nutrition_score',
#         'additives_n': 'nombre_additifs',
#         'additives_fr': 'additifs',
#         'quantity': 'quantité',
#         'energy_100g': 'energie_100g',
#         'proteins_100g': 'proteines_100g',
#         'sugars_100g': 'sucres_100g',
#         'fiber_100g': 'fibres_100g',
#         'salt_100g': 'sel_100g'

#     }
#     dataframe.rename(columns=column_name_mapping, inplace=True)

#     return dataframe

In [ ]:
# df_filtered = clean_column_names(df_filtered)